### Import libraries

In [1]:
import numpy as np

from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

from kerastuner import RandomSearch

In [2]:
TRIALS = 3  # number of models to train
EPOCHS = 2  # number of epoch per model

### Get the MNIST dataset.

In [3]:
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train = np.expand_dims(x_train.astype('float32') / 255, -1)
x_val = np.expand_dims(x_val.astype('float32') / 255, -1)
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)

In [4]:
def build_model(hp):
    """Function that build a TF model based on hyperparameters values.
    Args:
        hp (HyperParameter): hyperparameters values
    Returns:
        Model: Compiled model
    """

    num_layers = hp.Int('num_layers', 2, 8, default=6)
    lr = hp.Choice('learning_rate', [1e-3, 5e-4])

    inputs = layers.Input(shape=(28, 28, 1))
    x = inputs

    for idx in range(num_layers):
        idx = str(idx)

        filters = hp.Int('filters_' + idx, 32, 256, step=32, default=64)
        x = layers.Conv2D(filters=filters, kernel_size=3, padding='same',
                          activation='relu')(x)

        # add a pooling layers if needed
        if x.shape[1] >= 8:
            pool_type = hp.Choice('pool_' + idx, values=['max', 'avg'])
            if pool_type == 'max':
                x = layers.MaxPooling2D(2)(x)
            elif pool_type == 'avg':
                x = layers.AveragePooling2D(2)(x)

    x = layers.Flatten()(x)
    outputs = layers.Dense(10, activation='softmax')(x)

    # Build model
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=Adam(lr),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [5]:
# Initialize the tuner by passing the `build_model` function
# and specifying key search constraints: maximize val_acc (objective),
# and the number of trials to do. More efficient tuners like UltraBand() can
# be used.
tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=TRIALS,
                     project_name='MNIST_results')

### Display search space overview

In [6]:
tuner.search_space_summary()

### Perform the model search. The search function has the same signature as `model.fit()`.

In [7]:
tuner.search(x_train, y_train, batch_size=128, epochs=2,
             validation_data=(x_val, y_val))

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 138s 2ms/sample - loss: 0.1708 - accuracy: 0.9449 - val_loss: 0.0405 - val_accuracy: 0.9871
Epoch 2/2
60000/60000 [==============================] - 126s 2ms/sample - loss: 0.0467 - accuracy: 0.9858 - val_loss: 0.0420 - val_accuracy: 0.9862


Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 82s 1ms/sample - loss: 0.2018 - accuracy: 0.9362 - val_loss: 0.0481 - val_accuracy: 0.9850
Epoch 2/2
60000/60000 [==============================] - 102s 2ms/sample - loss: 0.0506 - accuracy: 0.9842 - val_loss: 0.0295 - val_accuracy: 0.9903


Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 72s 1ms/sample - loss: 0.2582 - accuracy: 0.9219 - val_loss: 0.0685 - val_accuracy: 0.9785
Epoch 2/2
60000/60000 [==============================] - 64s 1ms/sample - loss: 0.0652 - accuracy: 0.9801 - val_loss: 0.0453 - val_accuracy: 0.9848


### Display the best models, their hyperparameters, and the resulting metrics.

In [8]:
tuner.results_summary()

### Retrieve the best model and display its architecture

In [9]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 128)       1280      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 160)         92320     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 96)          138336